In [2]:
import pandas as pd
import numpy as np
import math
from numpy.linalg import norm
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

import warnings
warnings.filterwarnings("ignore")

In [3]:
## Calling the dataframe
data = pd.read_csv("main_final_recommender_dataset.csv")

In [4]:
data.head()

,college,course,code,fees,location,cao_score,interest,job_domain,personality
0,Technological University Dublin,TU059 Computer Science (Data Science),TU059,7000,Dublin,304,Applications Developer,IT,serious
1,University College Dublin,DN150 Biomedical Engineering,DN150,7574,Dublin,551,Business Analyst,Non_IT,extraverted
2,University College Dublin,NQS2 Chemical & Bioprocess Engineering,NQS2,7574,Dublin,224,Banker,Non_IT,responsible
3,University College Dublin,NVS1 Civil Engineering,NVS1,7574,Dublin,313,Human Resources,Non_IT,dependable
4,University College Dublin,DN201 Computer Science,DN201,7574,Dublin,497,Business Analyst,IT,lively


In [5]:
## making the buckets for the Budget
## 1: € 2000- € 4000 2: € 4000 - € 8000 3: € 8000 - € 10000 4: € 10000 - € 20000 Making in the Increasing Order
def budget(num):
  if num>=2000 and num<4000:
    return 1
  elif num>=4000 and num<8000:
    return 2
  elif num>=8000 and num<10000:
    return 3
  else:
    return 4

bud_value = data['fees'].values
bucket_bud = []
for num in bud_value:
  bucket_bud.append(budget(num))

data['fees_Bucket'] = bucket_bud


In [6]:
cols = list(data.columns.values)
cat_cols = ["college","course","code","location","interest","job_domain","personality"]
num_cols = ["cao_score"]
data1 = data.copy()

le_p = preprocessing.LabelEncoder()
le_l = preprocessing.LabelEncoder()
le_i = preprocessing.LabelEncoder()
le_jd = preprocessing.LabelEncoder()
le_bd = preprocessing.LabelEncoder()

data1['fees_Bucket'] = data1['fees_Bucket'].astype('str')

data1["location"] = le_l.fit_transform(data1["location"])
for elem in data1['location'].unique():
    data1['location'+str(elem)] = data1['location'] == elem

data1["interest"] = le_i.fit_transform(data1["interest"])
for elem in data1['interest'].unique():
    data1['interest'+str(elem)] = data1['interest'] == elem

data1["job_domain"] = le_jd.fit_transform(data1["job_domain"])
for elem in data1['job_domain'].unique():
    data1['job_domain'+str(elem)] = data1['job_domain'] == elem

#data1["Budget_Bucket"] = data1["Budget_Bucket"])
for elem in data1['fees_Bucket'].unique():
    data1['fees'+str(elem)] = data1['fees_Bucket'] == elem

data1["personality"] = le_p.fit_transform(data1["personality"])
for elem in data1['personality'].unique():
    data1['personality'+str(elem)] = data1['personality'] == elem

for num in num_cols:
  data1[num] = data1[num].astype('int')

rank_nor = []
maxi = 600.0
mini = 0.0
for num in data1['cao_score'].values:
  rank_nor.append(float((num - mini)/(maxi - mini)))

data1['cao_score'] = rank_nor

In [7]:
data_input = pd.DataFrame()
cao_points = input("Enter the CAO Points: [Note: Max 600 and min 0 is acceptable] - ")
city_name = input("Enter the City Name from the Drop Down: ")
field_interest = input("Enter the filed of Job interest from the choice: ")
hobbies = input("Enter your career interest: ")
spending_limit = input("Enter the fees: ")
personality = input("Enter the kind of person from the drop down: ")

data_sim = data1.copy()

data_input['cao_Score'] = [float((int(cao_points)-mini)/(maxi-mini))]
data_input['fees_Bucket'] = [budget(int(spending_limit))]
data_input['location'] = [city_name]
data_input['interest'] = [hobbies]
data_input['job_domain'] = [field_interest]
data_input['personality'] = [personality]

data_input['location'] = le_l.transform(data_input['location'])
for elem in data1['location'].unique():
    data_input['location'+str(elem)] = data_input['location'] == elem
data_input['interest'] = le_i.transform(data_input['interest'])
for elem in data1['interest'].unique():
    data_input['interest'+str(elem)] = data_input['interest'] == elem
data_input['job_domain'] = le_jd.transform(data_input['job_domain'])
for elem in data1['job_domain'].unique():
    data_input['job_domain'+str(elem)] = data_input['job_domain'] == elem
data_input["personality"] = le_p.fit_transform(data_input["personality"])
for elem in data1['personality'].unique():
    data_input['personality'+str(elem)] = data_input['personality'] == elem
data_input['fees_Bucket'] = data_input['fees_Bucket'].astype('str')
for elem in data1['fees_Bucket'].unique():
    data_input['fees_budget'+str(elem)] = data_input['fees_Bucket'] == elem

for num in ['code','college', 'course', 'fees', 'location', 'interest', 'job_domain', 'fees_Bucket']:
  data_sim.pop(num)
  try:
    data_input.pop(num)
  except:
    continue


pairwise_similarities=np.dot(data_sim.values,data_input.T)/(norm(data_sim.values)*norm(data_input.values))

Enter the CAO Points: [Note: Max 600 and min 0 is acceptable] - 500
Enter the City Name from the Drop Down: Dublin
Enter the filed of Job interest from the choice: IT
Enter your career interest: Banker
Enter the fees: 7000
Enter the kind of person from the drop down: serious


In [8]:
data["similarities"] = pairwise_similarities
data_final = data[['college', 'course','code', 'fees', 'similarities']]
final_df = data_final.sort_values(by=['similarities'], ascending=False)

In [9]:
print("--------------------------------------------------------------------------------------------------")
print("------------------------------- Recommendation 1 -------------------------------------------------")
print("College Name : ", np.asscalar(final_df.head(1)["college"].values))
print("Course Name  : ", np.asscalar(final_df.head(1)["course"].values))
print("Code         : ", np.asscalar(final_df.head(1)["code"].values))
print("Budget       : ", np.asscalar(final_df.head(1)["fees"].values))
print("--------------------------------------------------------------------------------------------------")
print("------------------------------- Recommendation 2 -------------------------------------------------")
print("College Name : ", np.asscalar(final_df.head(2).tail(1)["college"].values))
print("Course Name  : ", np.asscalar(final_df.head(2).tail(1)["course"].values))
print("Code         : ", np.asscalar(final_df.head(2).tail(1)["code"].values))
print("Budget       : ", np.asscalar(final_df.head(2).tail(1)["fees"].values))
print("--------------------------------------------------------------------------------------------------")
print("------------------------------- Recommendation 3 -------------------------------------------------")
print("College Name : ", np.asscalar(final_df.head(3).tail(1)["college"].values))
print("Course Name  : ", np.asscalar(final_df.head(3).tail(1)["course"].values))
print("Code         : ", np.asscalar(final_df.head(3).tail(1)["code"].values))
print("Budget       : ", np.asscalar(final_df.head(3).tail(1)["fees"].values))
print("--------------------------------------------------------------------------------------------------")
print("------------------------------- Recommendation 4 -------------------------------------------------")
print("College Name : ", np.asscalar(final_df.head(4).tail(1)["college"].values))
print("Course Name  : ", np.asscalar(final_df.head(4).tail(1)["course"].values))
print("Code         : ", np.asscalar(final_df.head(4).tail(1)["code"].values))
print("Budget       : ", np.asscalar(final_df.head(4).tail(1)["fees"].values))
print("--------------------------------------------------------------------------------------------------")
print("------------------------------- Recommendation 5 -------------------------------------------------")
print("College Name : ", np.asscalar(final_df.head(5).tail(1)["college"].values))
print("Course Name  : ", np.asscalar(final_df.head(5).tail(1)["course"].values))
print("Code         : ", np.asscalar(final_df.head(5).tail(1)["code"].values))
print("Budget       : ", np.asscalar(final_df.head(5).tail(1)["fees"].values))
print("--------------------------------------------------------------------------------------------------")

--------------------------------------------------------------------------------------------------
------------------------------- Recommendation 1 -------------------------------------------------
College Name :  Trinity College Dublin
Course Name  :  TR032 Computer engineering
Code         :  TR032
Budget       :  7332
--------------------------------------------------------------------------------------------------
------------------------------- Recommendation 2 -------------------------------------------------
College Name :  Griffith College
Course Name  :  Audio and Music Technology
Code         :  GA1112
Budget       :  8950
--------------------------------------------------------------------------------------------------
------------------------------- Recommendation 3 -------------------------------------------------
College Name :  National College of Ireland
Course Name  :  NC003 Computing
Code         :  NC003
Budget       :  3080
------------------------------------------